In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra
import math
from pyroomacoustics.doa import spher2cart
import csv
from itertools import combinations
import scipy.io as sio

In [2]:
def getData(source_name):
    """ Returns the data depending on the cycle number related and the specific sound source.
    
        Keyword arugments:
        
        source_name -- specifies each sound cycle 
    
    """
    
    # Create a dictionary with all cycles 
    source_name_dict = dict({'S1_Cycle1': ['S1/S1_Cycle1', 'S1'],
                             'S1_Cycle2': ['S1/S1_Cycle2', 'S1'] #,
#                              'S1_Cycle3': ['S1/S1_Cycle3', 'S1'],
#                              'S1_Cycle4': ['S1/S1_Cycle4', 'S1'],
#                              'S1_Cycle5': ['S1/S1_Cycle5', 'S1'],
#                              'S1_Cycle6': ['S1/S1_Cycle6', 'S1'],
#                              'S1_Cycle7': ['S1/S1_Cycle7', 'S1'],
#                              'S1_Cycle8': ['S1/S1_Cycle8', 'S1'],
#                              'S1_Cycle9': ['S1/S1_Cycle9', 'S1'],
#                              'S1_Cycle10': ['S1/S1_Cycle10', 'S1'],
#                              'S1_Cycle11': ['S1/S1_Cycle11', 'S1'],
#                              'S1_Cycle12': ['S1/S1_Cycle12', 'S1'],
#                              'S1_Cycle13': ['S1/S1_Cycle13', 'S1'],
#                              'S1_Cycle14': ['S1/S1_Cycle14', 'S1'],
#                              'S1_Cycle15': ['S1/S1_Cycle15', 'S1'],
#                              'S1_Cycle16': ['S1/S1_Cycle16', 'S1'],
#                              'S1_Cycle17': ['S1/S1_Cycle17', 'S1'],
#                              'S1_Cycle18': ['S1/S1_Cycle18', 'S1'],
#                              'S1_Cycle19': ['S1/S1_Cycle19', 'S1'],
#                              'S1_Cycle20': ['S1/S1_Cycle20', 'S1'],
#                              'S1_Cycle21': ['S1/S1_Cycle21', 'S1'],
#                              'S1_Cycle22': ['S1/S1_Cycle23', 'S1'],
#                              'S1_Cycle23': ['S1/S1_Cycle23', 'S1'],
#                              'S2_Cycle1': ['S1/S1_Cycle1', 'S2'],
#                              'S2_Cycle2': ['S1/S1_Cycle2', 'S2'],
#                              'S2_Cycle3': ['S1/S1_Cycle3', 'S2'],
#                              'S2_Cycle4': ['S1/S1_Cycle4', 'S2'],
#                              'S2_Cycle5': ['S1/S1_Cycle5', 'S2'],
#                              'S2_Cycle6': ['S1/S1_Cycle6', 'S2'],
#                              'S2_Cycle7': ['S1/S1_Cycle7', 'S2'],
#                              'S2_Cycle8': ['S1/S1_Cycle8', 'S2'],
#                              'S2_Cycle9': ['S1/S1_Cycle9', 'S2'],
#                              'S2_Cycle10': ['S1/S1_Cycle10', 'S2'],
#                              'S2_Cycle11': ['S1/S1_Cycle11', 'S2'],
#                              'S2_Cycle12': ['S1/S1_Cycle12', 'S2'],
#                              'S2_Cycle13': ['S1/S1_Cycle13', 'S2'],
#                              'S2_Cycle14': ['S1/S1_Cycle14', 'S2'],
#                              'S2_Cycle15': ['S1/S1_Cycle15', 'S2'],
#                              'S2_Cycle16': ['S1/S1_Cycle16', 'S2'],
#                              'S2_Cycle17': ['S1/S1_Cycle17', 'S2'],
#                              'S2_Cycle18': ['S1/S1_Cycle18', 'S2'],
#                              'S2_Cycle19': ['S1/S1_Cycle19', 'S2'],
#                              'S2_Cycle20': ['S1/S1_Cycle20', 'S2'],
#                              'S2_Cycle21': ['S1/S1_Cycle21', 'S2'],
#                              'S2_Cycle22': ['S1/S1_Cycle23', 'S2'],
#                              'S2_Cycle23': ['S1/S1_Cycle23', 'S2']
                            })
    for key in source_name_dict.keys():
        if source_name == key:
            data = sio.loadmat(source_name_dict[key][0])
            sound_data = data[source_name_dict[key][1]]
    
    return sound_data

In [3]:
def mic_run(data, *args):
    
    """ Returns each of the 3 microphone locations and the signals list corresponding to the specific microphone.
        Note: The microphone locations are under a new coordinate system in relation to the center of the box
              (whose center = [(0.34925/2),(0.219964/2),(0.2413/2)] is the origin)
    
        Keyword arguments:
        data -- the signal associated with each microphone
        args -- list of the microphones 
    
    """
    # Empty Lists
    signal_list = []
    mic_location = []
    
    # Dictionary of the microphone locations and their respective signals
    microphones_locations_dict = dict({
        'mic1': [[-0.102235, -0.109982, 0.056388],  data[0]],  #mic 1 # location of channel
        'mic2': [[-0.102235, -0.109982, 0.001524],  data[1]],
        'mic3': [[-0.102235, -0.109982, -0.053340], data[2]], 
        'mic4': [[-0.102235, -0.109982, -0.108204], data[3]],
        'mic5': [[-0.052197, -0.109982, 0.056388], data[4]],
        'mic6': [[-0.052197, -0.109982, 0.001524], data[5]],
        'mic7': [[-0.052197, -0.109982, -0.053340], data[6]],
        'mic8': [[-0.052197, -0.109982, -0.108204], data[7]],
        'mic9': [[-0.027304, -0.109982, 0.056388], data[8]],
        'mic10': [[-0.027304, -0.109982, 0.001524], data[9]],
        'mic11': [[-0.027304, -0.109982, -0.053340], data[10]],
        'mic12': [[-0.027304, -0.109982, -0.108204], data[11]]
        }) 
    
    
    # Look for a match between the dictionary of microphone locations and the microphone in the list
    for arg in args:
        for key in microphones_locations_dict.keys():
            if arg == key:
                # Record the location
                mic_location.append(microphones_locations_dict[key][0])
                
                # Record the signal
                signal_list.append(microphones_locations_dict[key][1]) 
    
    print(signal_list)
    return signal_list, mic_location
    

In [4]:
def centroid(*args):  
    
    """ Returns the center of n number of microphones. 
    
    Keyword Arguments:
    
    args -- location of each n microphone 
    
    """
    
    # Initiate 
    microphone_array = np.zeros((len(args), len(args[1])))
    
    # Converts micrphone locations into an array
    for i in range(len(args)):
        microphone_array[i, :] = np.array(args[i])
   
    # Finds the centroid
    return np.sum(microphone_array, axis = 0)/len(args)
   

In [5]:
def difference_of_arrivals(speed_sound,signal_list,algo_name,fs,nfft, *mic_location):

    """ Returns an azimuth and co-latitude for each pair of microphones. 

        Keyword arugments:

            sound_speed -- Specific speed of sound
            mic1_location -- location of microphone 1
            mic2_location -- location of microphone 2
            mic3_location -- location of microphone 3
            signal_list -- the microphone signals
            algo_name -- Specific distance of arrival (DOA) method
            fs -- sampling frequency
            nfft -- Fast Fourier Transform (FFT) size
            mic_location -- location of each microphone
    """

    #Add 3-microphone array in [x,y,z] order
    R = np.column_stack(list(zip(*mic_location)))
    
    print(signal_list)
    X = np.array([pra.stft(signal, nfft, nfft // 2,transform=np.fft.rfft).T for signal in signal_list])

    if sound_speed == 30.:
        freq_range = [0,250]
    if sound_speed == 1500.:
        freq_range = [0,250]

    # Construct the new DOA object
    doa = pra.doa.algorithms[algo_name](L=R, fs=fs, nfft=nfft, c=sound_speed, num_src=1, max_four=4,
                              dim=3,azimuth=np.linspace(-180.,180.,360)*np.pi/180,
                                 colatitude=np.linspace(-90.,90.,180)*np.pi/180)

    doa.locate_sources(X, freq_range=freq_range)

    #all in radians
    return doa.azimuth_recon, doa.colatitude_recon

In [6]:
def match(azimuth_recon_1,colatitude_recon_1,azimuth_recon_2, colatitude_recon_2, centroid_1,centroid_2):
    
    """ Returns the two closest points or the exact point of match using two pairs of azimuth and colatitude angles. 
    
        Keywords Arguments:
        
        azimuth_recon_1 -- the first reconstructed azimuth angle
        colatitude_recon_1 -- the first reconstructed colatitude angle
        azimuth_recon_2 -- the second reconstructed azimuth angle
        colatitude_recon_1 -- the second reconstructed colatitude angle
        centroid_1 -- the first centroid
        centroid_2 -- the second centroid
        
        """
    
   # Constants: tolerence factor and extremely large distance
    tol = 8e-2
    prevDist = 99999999
    
    # Initialize a list of radii to loop through
    for r1 in list(np.arange(0,5,tol)):
        for r2 in list(np.arange(0,5,tol)):
            
            # Get the estimates
            estimate_1 = np.array(spher2cart(r1,azimuth_recon_1,colatitude_recon_1)) + np.array(centroid_1[:,np.newaxis]) 
            estimate_2 = np.array(spher2cart(r2,azimuth_recon_2,colatitude_recon_2)) + np.array(centroid_2[:,np.newaxis]) 

            # Calculate the euclidean distance between them
            dist = np.array(np.sqrt(np.einsum('i...,i...', (estimate_1 - estimate_2), (estimate_1 - estimate_2)))[:,np.newaxis])

            # Faster way is to just compare to tables? 
            # Compare the distance to this tolerance
            if dist < tol: 
                if dist == 0:
                    return estimate_1, [], True
                else:
                    return estimate_1, estimate_2, False
            
            ## If the distance break out of the loop
            if dist > prevDist:
                prevDist = 9999999
                break
            prevDist = dist
            
    return [], [], False

In [7]:
def main(sound_speed,algo_name,sound_data,combinations_number):
    
    """ Returns list of points that are either close to the point or the exact point itself.
    
        Keyword arguments:
        
        sound_speed -- specific speed of sound
        algo_name -- specific distance of arrival (DOA) method to call
        sound_data -- specific data to perform the localizing
        combinations_number -- number of microphone
    """
    
    
    #Constants 
    fs = 16000  # sampling frequency
    nfft = 256  # FFT size
    
    mics = ['mic1','mic2','mic3','mic4','mic5','mic6','mic7','mic8','mic9','mic10','mic11','mic12']
    
    mic_list=list(combinations(mics,combinations_number))
    two_pair_mic_list_full=list(combinations(mic_list,2))
    
    # FULL Microphone pairing list
    #two_pair_mic_list = two_pair_mic_list_full
    
    # DEBUG PURPOSES:
    two_pair_mic_list = two_pair_mic_list_full[:15]
    #print(two_pair_mic_list)
    outputs_list = []
    

    for i in range(0,(len(two_pair_mic_list))):
        
        #Group 1
        # Multi-thread 1
        [signal_1, mic_locations_1] = mic_run(sound_data, *[two_pair_mic_list[i][0][j] for j in range(combinations_number)])
        
        # Multi-thread 2
        [signal_2, mic_locations_2] = mic_run(sound_data, *[two_pair_mic_list[i][1][j] for j in range(combinations_number)])
        
        
        # Group 2
        # Mutli-thread 3
        centroid_1 = centroid(*mic_locations_1)
        # Mutli-thread 4
        centroid_2 = centroid(*mic_locations_2)

        # Group 3
        # Multi-thread 5
        [azimuth_recon_1, colatitude_recon_1] = difference_of_arrivals(sound_speed,signal_1,algo_name,fs,nfft,*mic_locations_1)
        # Multi-thread 6
        [azimuth_recon_2, colatitude_recon_2] = difference_of_arrivals(sound_speed,signal_2,algo_name,fs,nfft,*mic_locations_2)
        
        # Group 4
        # Multi-thread 7
        output1, output2, isMatch = match(azimuth_recon_1,colatitude_recon_1,azimuth_recon_2,colatitude_recon_2, centroid_1,centroid_2)
        
        if isMatch:
            outputs_list = output1
            break
        
        elif np.prod(output1.shape) > 0 or output1[output != np.nan]: #output1 != None and output1 != []:
            outputs_list.append(output1)
            outputs_list.append(output2)

    return outputs_list

In [8]:
if __name__ == '__main__':
    c = [30., 1500.]    # speed of sound
    combinations_number = 3

    # DEBUG PURPOSES:
    sound_list = ['S1_Cycle1','S1_Cycle2'] #, 'S2_Cycle1','S2_Cycle2']
    
    # Now we can test all the algorithms available
    algo_names = ['MUSIC','WAVES','SRP','TOPS','CSSM']
    for source_name in sound_list:
        for sound_speed in c:
            for algo_name in algo_names:
                sound_data = getData(source_name)
                outputs_list = main(sound_speed,algo_name,sound_data,combinations_number)
                filename = str(source_name)+'_sound_source_localization_c'+str(sound_speed)+'_'+str(algo_name)+'_test.csv'
                with open(filename, mode='w') as sound_source_file:
                    writer = csv.writer(sound_source_file,delimiter=',')
                    writer.writerow(['X Location', 'Y Location', 'Z Location'])
                    writer.writerows(outputs_list)

                sound_source_file.close()

[array([-0.0187686 , -0.01998932, -0.0187686 , ...,  0.0062562 ,
        0.00808728,  0.00961318]), array([0.01724269, 0.01724269, 0.01815824, ..., 0.01022354, 0.01144427,
       0.01388571]), array([-0.02456703, -0.02792401, -0.02914473, ...,  0.09231708,
        0.08529793,  0.08255131])]
[array([-0.0187686 , -0.01998932, -0.0187686 , ...,  0.0062562 ,
        0.00808728,  0.00961318]), array([0.01724269, 0.01724269, 0.01815824, ..., 0.01022354, 0.01144427,
       0.01388571]), array([-0.01815824, -0.01968414, -0.01785306, ...,  0.01113909,
        0.01205463,  0.01419089])]
[array([-0.0187686 , -0.01998932, -0.0187686 , ...,  0.0062562 ,
        0.00808728,  0.00961318]), array([0.01724269, 0.01724269, 0.01815824, ..., 0.01022354, 0.01144427,
       0.01388571]), array([-0.02456703, -0.02792401, -0.02914473, ...,  0.09231708,
        0.08529793,  0.08255131])]


KeyboardInterrupt: 